This notebook is to demostrate how to use groqon for tool usage or function calling.


groqon : [https://www.github.com/tikendraw/groq-on](https://www.github.com/tikendraw/groq-on)


In [31]:
from dotenv import load_dotenv
from groq import Groq
import json

True

In [32]:
# a function to convert dates to years
def days_to_years_func(days: int = 0) -> float:
    """converts days into years"""
    return days / 365


def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:  # noqa
        return json.dumps({"error": "Invalid expression"})

In [33]:
client2 = Groq(
    api_key="not needed",
    base_url="http://localhost:8080/",  # match the port used by groqon sever
    timeout=30,
)
MODEL = "llama3-groq-70b-8192-tool-use-preview"
# MODEL='llama3-8b-8192'
# MODEL='llama-3.1-70b-versatile'

In [35]:
def run_conversation(user_prompt, client=client2):
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results.",
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "days_to_years_func",
                "description": "converts days to years",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "day": {
                            "type": "int",
                            "description": "number of days, e.g. 245",
                        }
                    },
                    "required": ["days"],
                },
            },
        },
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=500,
        temperature=0.88,
        top_p=0.77,
        stream=False,
    )
    print(response)
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
            "days_to_years_func": days_to_years_func,
        }

        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = str(function_to_call(**function_args))
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(model=MODEL, messages=messages)
        print(second_response)
        return second_response.choices[0].message.content

In [36]:
# user_prompt = "calculate 3*25-55*2/25-4-2?"
user_prompt = "i have been running for  4312 days , what would that be in years"
response = run_conversation(user_prompt)

 respnse in cnnnectin pool <Response [200] content self.content stream <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x774874ce6e50>>
 respnse in cnnnectin pool steam <httpcore._sync.http11.HTTP11ConnectionByteStream object at 0x774874ce6e50>
ChatCompletion(id='chatcmpl-4f79e443-3754-4daf-99e7-b9f017ae8de1', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4n9n', function=Function(arguments='{"days": 4312}', name='days_to_years_func'), type='function')]))], created=1722500368, model='llama3-groq-70b-8192-tool-use-preview', object='chat.completion', system_fingerprint='fp_ee4b521143', usage=CompletionUsage(completion_tokens=29, prompt_tokens=309, total_tokens=338, completion_time=0.087751781, prompt_time=0.022203729, queue_time=None, total_time=0.10995551), x_groq={'id': 'req_01j46gbj5dedz8gxw58x8g964p'})
 respnse in cnnnec

In [37]:
response

'You have been running for approximately 11.81 years.'

In [38]:
days_to_years_func(4312)

11.813698630136987